In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Embedding, Flatten, Normalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import L1L2, L1, L2
from sklearn.preprocessing import StandardScaler
from matplotlib import gridspec

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Define base path for all data files
PATH = "/global/cfs/projectdirs/m4045/users/rhuang/T2K/onoffaxisdata/FormattedData_v13/"

# Load reconstructed Monte Carlo data
# Shape is (n_events, 14) where 14 represents:
# - 3 values for muon: (pmu, thetamu, phimu)
# - 3 values for proton: (ppr, thetapr, phipr)
# - 8 values for one-hot encoded sample index
mc_reco = np.load(PATH + "mc_vals_reco_Nominal.npy")

# Load event weights
# Shape is (n_events,) - Accounts for POT (Protons On Target)
# Sum of weights is approximately 32k
mc_weights = np.load(PATH + "mc_weights_reco_Throw0_MCStat0.npy")

# Load fake data study weights
# Shape is (n_events,) - Sampled from Poisson distribution (mostly zeros)
# Sum is approximately 25k
fake_data_weights = np.load(PATH + "mc_weights_reco_FakeDataStudy6_BootstrapV2StatThrow0.npy")

# Note: Commented out as shape doesn't match n_events (is ~2.88 times larger)
# pass_truth = np.load(DATA_PATH + "mc_pass_truth_Nominal.npy")

# Create masks for data selection (currently selecting all events)
data_mask = np.ones_like(mc_weights, dtype=bool)
mc_mask = np.ones_like(mc_weights, dtype=bool)

# Prepare data for machine learning
# Combine MC reconstructed data for both masks
combined_features = np.concatenate([
    mc_reco[mc_mask, :],
    mc_reco[data_mask, :]
])

# Create binary labels: 0 for MC, 1 for data
combined_labels = np.concatenate([
    np.zeros(len(mc_reco[mc_mask, :])),
    np.ones(len(mc_reco[data_mask, :]))
])

# Combine weights
combined_weights = np.concatenate([
    mc_weights[mc_mask],
    fake_data_weights[data_mask]
])

# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    combined_features,
    combined_labels,
    combined_weights,
    test_size=0.2
)

# Stack labels and weights together
# This creates arrays of shape (n_samples, 2) where:
# - Column 0 contains the binary label (0 for MC, 1 for data)
# - Column 1 contains the event weight
y_train_with_weights = np.stack((y_train, w_train), axis=1)
y_test_with_weights = np.stack((y_test, w_test), axis=1)

In [ ]:
def weighted_binary_crossentropy(y_true, y_pred):
    # Extract weights and true labels
    weights = y_true[:, 1:2]  # event weights
    y_true = y_true[:, 0:1]   # actual labels for loss
    
    # Clip predictions to prevent numerical issues
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1 - epsilon)
    
    # Calculate weighted binary cross-entropy
    binary_cross_entropy = y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred)
    weighted_loss = -weights * binary_cross_entropy
    
    return K.mean(weighted_loss)

In [ ]:
# Define model parameters
FEATURE_INDICES = range(14)  # Using all 14 features
HIDDEN_UNITS = 100
DROPOUT_RATE = 0  # Note: this is effectively not using dropout
ACTIVATION = 'relu'
LEARNING_RATE = 5e-5

# Build model with cleaner structure
def build_model(feature_indices, hidden_units, dropout_rate, activation):
    """
    Build a neural network model for binary classification with weighted loss.
    
    Args:
        feature_indices: Indices of features to use
        hidden_units: Number of neurons in each hidden layer
        dropout_rate: Dropout rate for regularization
        activation: Activation function for hidden layers
        
    Returns:
        Compiled Keras model
    """
    inputs = Input((len(feature_indices),))
    
    x = inputs
    # Create multiple identical hidden layers in a loop
    for _ in range(4):  # 4 hidden layers
        x = Dense(hidden_units, activation=activation, kernel_initializer='GlorotUniform')(x)
        x = Dropout(dropout_rate)(x)
    
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss=weighted_binary_crossentropy, optimizer=optimizer)
    
    return model

# Create model
model = build_model(FEATURE_INDICES, HIDDEN_UNITS, DROPOUT_RATE, ACTIVATION)

# Prepare data for training
X_train_features = X_train[:, FEATURE_INDICES]
X_test_features = X_test[:, FEATURE_INDICES]

# Define callbacks for training
callbacks = [
    EarlyStopping(patience=20, verbose=False, restore_best_weights=True),
    ReduceLROnPlateau(patience=10, min_lr=1e-6, factor=0.3)
]

# Train model
history = model.fit(
    X_train_features, y_train_with_weights,
    epochs=150,
    batch_size=1024,
    validation_data=(X_test_features, y_test_with_weights),
    callbacks=callbacks,
    verbose=0
)

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label="Training loss")
plt.plot(history.history['val_loss'], label="Validation loss")
plt.legend()
plt.title("Model Training History")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()

In [ ]:
# Calculate reweighting factors using the trained model
reweight_factors = model.predict(mc_reco, batch_size=8192)[:,0]
reweight_factors = reweight_factors / (1.0 - reweight_factors)

In [ ]:
plt.hist(model.predict(mc_reco, batch_size=8192)[:,0])

In [ ]:
_ = plt.hist(reweight_factors, bins=40, range=[0.5,1.5])

In [ ]:
# Plot the comparison of distributions
plt.figure(figsize=(6, 4))

# Create histograms for comparison
plt.hist(
    mc_reco[:, 0], 
    weights=fake_data_weights, 
    bins=np.linspace(-4, 4, 30), 
    alpha=0.5,
    label="Data"
)

plt.hist(
    mc_reco[:, 0], 
    weights=mc_weights, 
    histtype='step', 
    linewidth=2,
    bins=np.linspace(-4, 4, 30), 
    label="Original MC"
)

plt.hist(
    mc_reco[:, 0], 
    weights=mc_weights * reweight_factors, 
    histtype='step', 
    linewidth=2,
    bins=np.linspace(-4, 4, 30), 
    label="ReWeight MC (Step 1)"
)

plt.xlabel("Muon Momentum (Standardized)")
plt.ylabel("Events / Bin")
plt.title("Comparison of Muon Momentum Distributions")
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

In [ ]:
# Plot the comparison of distributions
plt.figure(figsize=(6, 4))

# Create histograms for comparison
plt.hist(
    mc_reco[:, 1], 
    weights=fake_data_weights, 
    bins=np.linspace(-4, 4, 30), 
    alpha=0.5,
    label="Data"
)

plt.hist(
    mc_reco[:, 1], 
    weights=mc_weights, 
    histtype='step', 
    linewidth=2,
    bins=np.linspace(-4, 4, 30), 
    label="Original MC"
)

plt.hist(
    mc_reco[:, 1], 
    weights=mc_weights * reweight_factors, 
    histtype='step', 
    linewidth=2,
    bins=np.linspace(-4, 4, 30), 
    label="ReWeight MC (Step 1)"
)

plt.xlabel("Muon CosTheta (Standardized)")
plt.ylabel("Events / Bin")
plt.title("Comparison of Muon Momentum Distributions")
plt.legend(loc='upper left')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

In [ ]:
# Define the base path for truth data files
PATH = "/global/cfs/projectdirs/m4045/users/rhuang/T2K/onoffaxisdata/FormattedData_v13/"

# Load truth-level Monte Carlo data
mc_truth = np.load(PATH + "mc_vals_truth_Topology.npy")

# Load truth-level event weights
weightsTruth = np.load(PATH + "mc_weights_truth_Throw0_MCStat0.npy")

# Load truth-level selection mask
pass_truth = np.load(PATH + "mc_pass_truth_Nominal.npy")

# Prepare features, labels, and weights for the second model
# - Concatenate the same truth data twice (once with original weights, once with reweighted)
# - Labels: 0 for original weights, 1 for reweighted
# - Split into train/test sets (80/20 split)
X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(
    np.concatenate([
        mc_truth[pass_truth, :],
        mc_truth[pass_truth, :]
    ]),
    np.concatenate([
        np.zeros(len(mc_truth[pass_truth, :])),
        np.ones(len(mc_truth[pass_truth, :]))
    ]),
    np.concatenate([
        weightsTruth[pass_truth],
        weightsTruth[pass_truth] * reweight_factors
    ]),
    test_size=0.2
)

# Stack labels and weights together for the training and test sets
# Column 0: binary label (0 for original, 1 for reweighted)
# Column 1: event weight
Y_train_1 = np.stack((Y_train_1, w_train_1), axis=1)
Y_test_1 = np.stack((Y_test_1, w_test_1), axis=1)

In [ ]:
# Reuse existing model structure for second classifier
# Define model parameters for the second model
FEATURE_INDICES_2 = range(11)  # Using 11 features for the second model
HIDDEN_UNITS_2 = 100           # Same as first model
DROPOUT_RATE_2 = 0             # No dropout, same as first model
ACTIVATION_2 = 'relu'          # Same activation
LEARNING_RATE_2 = 5e-5         # Same learning rate

# Create second model using the same build_model function
mymodel = build_model(FEATURE_INDICES_2, HIDDEN_UNITS_2, DROPOUT_RATE_2, ACTIVATION_2)

# Prepare data for training
trainingList = X_train_1[:, FEATURE_INDICES_2]
testList = X_test_1[:, FEATURE_INDICES_2]

# Define callbacks for second model training (with longer patience)
callbacks_2 = [
    EarlyStopping(patience=20, verbose=False, restore_best_weights=True),
    ReduceLROnPlateau(patience=10, min_lr=1e-6, factor=0.3)
]

# Train the second model
hist_s2 = mymodel.fit(
    trainingList, Y_train_1,
    epochs=150,
    batch_size=1024,
    validation_data=(testList, Y_test_1),
    callbacks=callbacks_2,
    verbose=0
)

In [ ]:
plt.plot(hist_s2.history['loss'], label="Training loss")
plt.plot(hist_s2.history['val_loss'], label="Test loss")
plt.legend()
plt.title("Step 2 Training")
plt.ylabel("Loss")
plt.xlabel("Epoch")

In [ ]:
reweightsTruth = mymodel.predict(mc_truth, batch_size=8192)[:,0]
reweightsTruth = reweightsTruth / (1. - reweightsTruth)

In [ ]:
prior=plt.hist(mc_truth[pass_truth,0], weights=weightsTruth[pass_truth], bins=np.linspace(-4,4,30), label="Truth Prior")
pull=plt.hist(mc_truth[pass_truth,0], weights=weightsTruth[pass_truth]*reweight_factors, histtype='step',bins=np.linspace(-4,4,30), label="Pulled Truth")
result=plt.hist(mc_truth[pass_truth,0], weights=weightsTruth[pass_truth]*reweightsTruth[pass_truth], histtype='step',bins=np.linspace(-4,4,30), label="Omnifold Step 2 Result")
plt.xlabel("Muon Momentum (Standardized)")
plt.legend(loc='upper right')